## This notebook is meant to accompany the trial_NGC2992.py file that is included in this directory with additional comments. 

### This analysis can produce $\sim 900$ GB of data so be sure that there is enough storage on your computer.

In this notebook, we will go through the code to produce Figures 4 & 5 of the associated BAT survey paper. This example outlines how to analyze BAT survey data to obtain a light curve/spectrum for an AGN, such as NGC 2992. 

First, we need to import our usual python packages.

In [ ]:
import glob
import os
import sys
import batanalysis as ba
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np
from astropy.time import Time, TimeDelta
from astropy.io import fits
from pathlib import Path
import swiftbat
import swiftbat.swutil as sbu
import pickle

We will also import pyXspec to be able to easily manipulate our fitted mosaic spectrum and generate the plot of the data versus the fit. 

In [ ]:
from xspec import *

Now we can query HEASARC to download data between Dec 15th 2004 and Dec 16th 2005 associated with BAT having the coordinates of NGC 2992 within its FOV and at least 1000 cm$^2$ of the detector plane is exposed to that point on the sky. 

In [ ]:
object_name='NGC2992'
object_location = swiftbat.simbadlocation(object_name)
object_batsource = swiftbat.source(ra=object_location[0], dec=object_location[1], name=object_name)

queryargs = dict(time="2004-12-15 .. 2005-12-16", fields='All', resultmax=0)
table_everything = ba.from_heasarc(**queryargs)

minexposure = 1000     # cm^2 after cos adjust
exposures = np.array([object_batsource.exposure(ra=row['RA'], dec=row['DEC'], roll=row['ROLL_ANGLE'])[0] for row in table_everything])
table_exposed = table_everything[exposures > minexposure]
print(f"Finding everything finds {len(table_everything)} observations, of which {len(table_exposed)} have more than {minexposure:0} cm^2 coded")

To download the data we would then do:
```
result = ba.download_swiftdata(table_exposed)
```

And to get the observation IDs for the successfully downloaded data we would do:

In [ ]:
obs_ids=[i for i in table_exposed['OBSID'] if result[i]['success']]

Remember, that when continuing a prior analysis, we do not need to requery the database to get the observation IDs. Instead, we can simply do:
```
obs_ids=[i.name for i in sorted(ba.datadir().glob("*")) if i.name.isnumeric()]
```

With the data downloaded and the list of observation IDs obtained, we can now process the survey observations by doing:

In [ ]:
noise_map_dir=Path("/Users/tparsota/Documents/PATTERN_MAPS/")
batsurvey_obs=ba.parallel.batsurvey_analysis(obs_ids, patt_noise_dir=noise_map_dir, nprocs=30)

With the survey dataset analyzed, we can fit the spectra and obtain upper limits for the nondetections. We set the photon index for the power law that is fit to the 5$\sigma$ upper limit spectrum to be 1.9 since this is what has been observed in the source in the past. 

Here, we set `nprocs = -2` so it will tell the python function to use 2 less than all the CPUs available on the users laptop. This is a convenient notation used by the python joblib package that can be used anywhere that nprocs is a value that can be specified. 

In [ ]:
batsurvey_obs=ba.parallel.batspectrum_analysis(batsurvey_obs, object_name,  ul_pl_index=1.9, use_cstat=True, nprocs=-2)

Now, we can view th elight curve showing our results by doing:

In [ ]:
fig, axes=ba.plot_survey_lc(batsurvey_obs, id_list= object_name, time_unit="UTC", values=["rate","snr", "flux", "PhoIndex", "exposure"])

Here, we see that we only have upper limits on the fluxes and thus do not have any detections. 

To try to get a detection, we can switch to looking at the AGN in monthly time bins. To do that, we follow the prescription outlined in our analysis of the Crab. We simply do:

In [ ]:
outventory_file=ba.merge_outventory(batsurvey_obs)
time_bins=ba.group_outventory(outventory_file, np.timedelta64(1, "M"), start_datetime=Time("2004-12-15"), end_datetime=Time("2005-12-16"))

mosaic_list, total_mosaic=ba.parallel.batmosaic_analysis(batsurvey_obs, outventory_file, time_bins, nprocs=3)


This produces our montly mosaics which we can then fit with spectra and plot those results on top of our individual survey dataset analysis to see if we get any detections on monthly time scales. The cell below does these steps.

In [ ]:
mosaic_list=ba.parallel.batspectrum_analysis(mosaic_list, object_name, ul_pl_index=1.9, use_cstat=True, nprocs=5)
total_mosaic=ba.parallel.batspectrum_analysis(total_mosaic, object_name, ul_pl_index=1.9, use_cstat=True, nprocs=1)

fig, axes=ba.plot_survey_lc([batsurvey_obs,mosaic_list], id_list= object_name, time_unit="UTC", values=["rate","snr", "flux", "PhoIndex", "exposure"], same_figure=True)


We can seee that the call to `plot_survey_lc` has a list of the batsurvey objects and the mosaics and we tell it to plot the two datasets simutaneously with the `same_figure` variable. 

Based on this plot, we still do not have a detection of the AGN. We can however take a look at our total time integrated mosaic and we will see that we do have a solid detection of the AGN. 

In [ ]:
fig, axes=ba.plot_survey_lc([batsurvey_obs,[total_mosaic]], id_list= object_name, time_unit="UTC", values=["rate","snr", "flux", "PhoIndex", "exposure"], same_figure=True)


Now, we will first save our data to create our light curve plots:

In [ ]:
all_data=ba.concatenate_data(batsurvey_obs, object_name, ["met_time", "utc_time", "exposure", "rate","rate_err","snr", "flux", "PhoIndex"])
with open('all_data_dictionary.pkl', 'wb') as f:
    pickle.dump(all_data, f)

all_data_monthly=ba.concatenate_data(mosaic_list, object_name, ["user_timebin/met_time", "user_timebin/utc_time", "user_timebin/met_stop_time", "user_timebin/utc_stop_time", "rate","rate_err","snr", "flux", "PhoIndex"])
with open('monthly_mosaic_dictionary.pkl', 'wb') as f:
    pickle.dump(all_data_monthly, f)


To create the light curve plots we now do:

In [ ]:
energy_range=None
time_unit="MET"
values=["rate","snr", "flux"]

survey_obsid_list=["all_data_dictionary", "monthly_mosaic_dictionary"]

obs_list_count=0
for observation_list in survey_obsid_list:

    with open(observation_list+".pkl", 'rb') as f:
        all_data=pickle.load(f)
        data=all_data[object_name]

    # get the time centers and errors
    if "mosaic" in observation_list:

        if "MET" in time_unit:
            t0 = TimeDelta(data["user_timebin/met_time"], format='sec')
            tf = TimeDelta(data["user_timebin/met_stop_time"], format='sec')
        elif "MJD" in time_unit:
            t0 = Time(data[time_str_start], format='mjd')
            tf = Time(data[time_str_end], format='mjd')
        else:
            t0 = Time(data["user_timebin/utc_time"])
            tf = Time(data["user_timebin/utc_stop_time"])
    else:
        if "MET" in time_unit:
            t0 = TimeDelta(data["met_time"], format='sec')
        elif "MJD" in time_unit:
            t0 = Time(data[time_str_start], format='mjd')
        else:
            t0 = Time(data["utc_time"])
        tf = t0 + TimeDelta(data["exposure"], format='sec')

    dt = tf - t0

    if "MET" in time_unit:
        time_center = 0.5 * (tf + t0).value
        time_diff = 0.5 * (tf - t0).value
    elif "MJD" in time_unit:
        time_diff = 0.5 * (tf - t0)
        time_center = t0 + time_diff
        time_center = time_center.value
        time_diff = time_diff.value

    else:
        time_diff = TimeDelta(0.5 * dt)  # dt.to_value('datetime')
        time_center = t0 + time_diff

        time_center = np.array([i.to_value('datetime64') for i in time_center])
        time_diff = np.array([np.timedelta64(0.5 * i.to_datetime()) for i in dt])

    x = time_center
    xerr = time_diff

    if obs_list_count == 0:
        fig, axes = plt.subplots(len(values), sharex=True) #, figsize=(10,12))

    axes_queue = [i for i in range(len(values))]
    # plot_value=[i for i in values]

    e_range_str = f"{14}-{195} keV"
    #axes[0].set_title(object_name + '; survey data from ' + e_range_str)

    for i in values:
        ax = axes[axes_queue[0]]
        axes_queue.pop(0)

        y = data[i]
        yerr = np.zeros(x.size)
        y_upperlim = np.zeros(x.size)

        label = i

        if "rate" in i:
            yerr = data[i + "_err"]
            label = "Count rate (cts/s)"
        elif i + "_lolim" in data.keys():
            # get the errors
            lolim = data[i + "_lolim"]
            hilim = data[i + "_hilim"]

            yerr = np.array([lolim, hilim])
            y_upperlim = data[i + "_upperlim"]

            # find where we have upper limits and set the error to 1 since the nan error value isnt
            # compatible with upperlimits
            yerr[:, y_upperlim] = 0.4 * y[y_upperlim]

        if "mosaic" in observation_list:
            if "weekly" in observation_list:
                zorder = 9
                c = "blue"
                m = "o"
                l="Weekly Mosaic"
                ms=5
                a=0.8
            else:
                zorder = 9
                c='green'
                m = "s"
                l = "Monthly Mosaic"
                ms=7
                a = 1
        else:
            zorder = 4
            c = "gray"
            m = "."
            l = "Survey Snapshot"
            ms=3
            a = 0.3

        ax.errorbar(x, y, xerr=xerr, yerr=yerr, uplims=y_upperlim, linestyle="None", marker=m, markersize=ms,
                    zorder=zorder, color=c, label=l, alpha=a)
                    
        #plt.gca().ticklabel_format(useMathText=True)
        ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))


        if ("flux" in i.lower()):
            ax.set_yscale('log')

        if ("snr" in i.lower()):
            ax.set_yscale('log')

        ax.set_ylabel(label)

    # if T0==0:
    if "MET" in time_unit:
        label_string = 'MET Time (s)'
    elif "MJD" in time_unit:
        label_string = 'MJD Time (s)'
    else:
        label_string = 'UTC Time (s)'

    axes[-1].set_xlabel(label_string)
    
    obs_list_count += 1


#add the UTC times as well
utc_time=Time(["2005-01-01", "2006-01-01"])
met_time=[]
for i in utc_time:
    met_time.append(sbu.datetime2met(i.datetime, correct=True))

for i,j in zip(met_time, utc_time.ymdhms):
    for ax in axes:
        ax.axvline(i, 0, 1, ls='--', color='k')
        if ax==axes[0]:
            ax.text(i, ax.get_ylim()[1]*1.03, f'{j["year"]}', fontsize=10, ha='center')

axes[1].set_ylabel("SNR")
axes[2].set_ylabel(r"Flux (erg/s/cm$^2$)")

axes[1].legend(loc= "lower center", ncol=2)

for ax, l in zip(axes, ["a","b","c","d"]):
    ax.text(0.01, .95, f"({l})", ha='left', va='top', transform=ax.transAxes,  fontsize=12)

fig.tight_layout()
plot_filename = object_name + '_survey_lc.pdf'
fig.savefig(plot_filename, bbox_inches="tight")


We can also now make a plot of our year long time-integrated spectrum and the best fit model. Here we use the saved xspec session info that was produced by BatAnalysis to load the xspec information related to the fit. This lets us get the model spectra in each energy bin once it has been folded through the response function.

In [ ]:
fig, ax=plt.subplots(1)
pha_file=total_mosaic.get_pha_filenames(id_list=object_name)[0]
emax=np.array(total_mosaic.emax)
emin=np.array(total_mosaic.emin)
ecen=0.5*(emin+emax)

os.chdir(pha_file.parent)

with fits.open(pha_file.name) as file:
    pha_data=file[1].data
    energies=file[-2].data

#get the xspec model info
mosaic_pointing_info=total_mosaic.get_pointing_info("mosaic", source_id=object_name)
xspec_session_name=mosaic_pointing_info['xspec_model'].name
flux=10**mosaic_pointing_info["model_params"]["lg10Flux"]["val"]
flux_err=10**np.array([mosaic_pointing_info["model_params"]["lg10Flux"]["lolim"], mosaic_pointing_info["model_params"]["lg10Flux"]["hilim"]])
flux_diff=np.abs(flux-flux_err)


phoindex=mosaic_pointing_info["model_params"]["PhoIndex"]["val"]
phoindex_err=np.array([mosaic_pointing_info["model_params"]["PhoIndex"]["lolim"], mosaic_pointing_info["model_params"]["PhoIndex"]["hilim"]])
phoindex_diff=np.abs(phoindex-phoindex_err)


xsp.Xset.restore(xspec_session_name)
xsp.Plot.device = "/null"
xsp.Plot("data resid")
energies = xsp.Plot.x()
edeltas = xsp.Plot.xErr()
rates = xsp.Plot.y(1,1)
errors = xsp.Plot.yErr(1,1)
foldedmodel = xsp.Plot.model()
dataLabels = xsp.Plot.labels(1)
residLabels = xsp.Plot.labels(2)

foldedmodel.append(foldedmodel[-1])
xspec_energy=total_mosaic.emin.copy()
xspec_energy.append(total_mosaic.emax[-1])
xspec_energy=np.array(xspec_energy)


ax.loglog(emin, pha_data['RATE'], color='k', drawstyle='steps-post')
ax.loglog(emax, pha_data['RATE'], color='k', drawstyle='steps-pre')
ax.errorbar(ecen, pha_data["RATE"], yerr=pha_data['STAT_ERR'], color='k', marker='None', ls='None', label=object_name+" 1 Year Mosaic Spectrum")
ax.set_ylabel("Count Rate (cts/s)", fontsize=14)
ax.set_xlabel("E (keV)", fontsize=14)
 
ax.tick_params(axis='both', which='major', labelsize=14)

l=f"Folded Model:\nFlux={flux/1e-11:-.3}$^{{{flux_diff[1]/1e-11:+.3}}}_{{{-1*flux_diff[0]/1e-11:+.3}}} \\times 10^{{-11}}$ erg/s/cm$^2$"+f"\n$\Gamma$={phoindex:-.3}$^{{{phoindex_diff[1]:+.2}}}_{{{-1*phoindex_diff[0]:+.2}}}$"
ax.loglog(xspec_energy, foldedmodel, color='r', drawstyle='steps-post', label=l)
ax.legend(loc='best')

fig.tight_layout()
fig.savefig(object_name+"_1year_spectrum.pdf")
